Let's get started sending some Spead data by importing the necessary libraries.

In [ ]:
import spead2
import spead2.send
import numpy as np

As with receiving, we need a `ThreadPool` in order to instantiate a stream. A `ThreadPool` is a collection of C threads, which allows the Spead2 library to send and receive data.

In [ ]:
thread_pool = spead2.ThreadPool()

The `spead2.send.StreamConfig` class handles is a struct-like object which encapsulates a users choice of stream parameters. The only thing we are going to change from the default value is the send rate, which we will set to `1e7`. The other defaults are: 
    `max_packet_size=1472`, 
    `rate=0.0`, 
    `burst_size=65535`, and 
    `max_heaps=4`. 
For our purposes, we can ignore these optional parameters. If you would like to dive deeper, however, you can read more at https://spead2.readthedocs.io/en/latest/py-send.html#spead2.send.StreamConfig

In [ ]:
stream_config = spead2.send.StreamConfig(rate=1e7)

We can now instantiate a `UdpStream` which we will use to send Spead data. We must pass in the `ThreadPool` to which it belongs, the Ipv4 address and the port we wish to send data to, and finally the `StreamConfig` object we just created. 

In [ ]:
stream = spead2.send.UdpStream(thread_pool, "10.0.1.112", 8888, stream_config)

The Spead protocol states that to send data, we create `Item` objects, which are accumulated together into `ItemGroups`, and finally chunked into manageable `Heaps` to be sent across the network. We now create a `spead2.send.ItemGroup`, to which we will add our data in the form of `Item`s. The `ItemGroup` will then expose some methods which allow us to easily create `Heap`s which we can send over our stream. 

In [ ]:
item_group = spead2.send.ItemGroup()

To add an `Item` to our `ItemGroup`, we also have to associate to this `Item` an `ItemDescriptor`. The `ItemDescriptor` contains the item's ID, name, description, shape, and Numpy data type. We can create an `ItemDescriptor` and add it to our `ItemGroup` all at once using the `ItemGroup.add_item` method.

In [ ]:
# this method adds an item descriptor to item_group, and returns the item to us
item = item_group.add_item( 0x1000, "number",   "just some large number", (),         dtype=np.uint32)
#                          item ID  item name   item description          itemShape   item data type

We can now add a value to the `Item` returned from `ItemGroup.add_item`. This change will automagically show up within our `ItemGroup`.

In [ ]:
item.value = 0xDEADBEEF

We now instantiate a `HeapGenerator` object, which takes in an `ItemGroup` and allows us to chunk our `ItemGroup` into heaps which we can then send over the network. A `HeapGenerator` will track the `ItemGroup` it is instantiated with, meaning that if you update your `ItemGroup` by adding anything to it or changing the values of any items within it, the `HeapGenerator` will include these changes in the next heap you generate. The `HeapGenerator` class is complex and has some optional parameters that you may want to read about here: https://spead2.readthedocs.io/en/latest/py-send.html?highlight=HeapGenerator#spead2.send.HeapGenerator

In [ ]:
heap_generator = spead2.send.HeapGenerator(item_group)

To send a heap with our `HeapGenerator` and `UdpStream` is pretty straightforward. Take a look:

In [ ]:
stream.send_heap(heap_generator.get_heap())

Once we are done sending all of our data and would like to let the receiver know that it can stop listening, we can send an end-of-heap marker. If you are using the Spead2 receiver from the other part of this tutorial, this will cause the receiver to stop listening for data and begin to print out what is has received. 

In [ ]:
stream.send_heap(heap_generator.get_end())

That's it for some simple sending. You now know the basics of the Spead2 API, and should you want to dive in deeper, can head over to https://spead2.readthedocs.io/en/latest/ to learn more about the advanced features the library has to offer.